In [ ]:
from abc import ABC, abstractmethod
from collections import defaultdict
import math

In [ ]:
class MCTS:
  "Monte Carlo tree searcher. First rollout, then choose a move."
  def __init__(self, c=1):
      self.Q = defaultdict(int)  # Total reward of each node
      self.N = defaultdict(int)  # Total visit count for each node
      self.children = dict()  # Children of each node
      self.c = c  # Exploration parameter

  def choose(self, node):
      "Choose the best child of the node."
      if node.is_terminal():
          raise RuntimeError(f"choose called on terminal node {node}")
      if node not in self.children:
          return node.find_random_child()

      def score(n):
          if self.N[n] == 0:
              return float("-inf")  # Avoid dividing by zero
          return self.Q[n] / self.N[n]

      return max(self.children[node], key=score)

  def do_rollout(self, node):
      "Perform a single playout from the given node."
      path = self._select(node)
      leaf = path[-1]
      self._expand(leaf)
      reward = self._simulate(leaf)
      self._backpropagate(path, reward)

  def _select(self, node):
      "Find an untried child of the node."
      path = []
      while True:
          path.append(node)
          if node not in self.children or not self.children[node]:
              return path
          unexplored = self.children[node] - set(self.children)
          if unexplored:
              n = unexplored.pop()
              path.append(n)
              return path
          node = self._uct_select(node)

  def _expand(self, node):
    "children에 node의 자식노드 추가"
    if node in self.children:
      return
    self.children[node] = node.find_children()

  def _simulate(self, node):
    "node의 무작위 시뮬레이션에 대한 결과 반환"
    invert_reward = True
    while True:
      if node.is_terminal():
        reward = node.reward()
        return 1 - reward if invert_reward else reward
      node = node.find_random_child()
      invert_reward = not invert_reward

  def _backpropagate(self, path, reward):
    "단말 노드의 조상 노드들에게 보상 전달"
    for node in reversed(path):
      self.N[node] += 1
      self.Q[node] += reward
      reward = 1 - reward

  def _uct_select(self, node):
    "탐험과 이용의 균형을 맞추어 node의 자식 노드 선택"
    assert all(n in self.children for n in self.children[node])
    log_N_vertex = math.log(self.N[node])

    def uct(n):
      "UCB(Upper confidence bound) 점수 계산"
      return self.Q[n] / self.N[n] + self.c * math.sqrt(2*log_N_vertex / self.N[n])

    return max(self.children[node], key=uct)

In [ ]:
class Node(ABC):
  "게임 트리의 노드로서 보드판의 상태 표현"
  @abstractmethod
  def find_children(self):
    return set()

  @abstractmethod
  def find_random_child(self):
    return None

  @abstractmethod
  def is_terminal(self):
    return True

  @abstractmethod
  def reward(self):
    return 0

  @abstractmethod
  def __hash__(self):
    return 123456789

  @abstractmethod
  def __eq__(node1, node2):
    return True

In [ ]:
from collections import namedtuple
from random import choice

TTTB = namedtuple("TicTacToeBoard", "tup turn winner terminal")

class TicTacToeBoard(TTTB): # Assuming Node is defined elsewhere
    @staticmethod
    def find_children(board):
        if board.terminal:
            return set()
        return {board.make_move(i) for i, value in enumerate(board.tup) if value is None}

    @staticmethod
    def find_random_child(board):
        if board.terminal:
            return None
        empty_spots = [i for i, value in enumerate(board.tup) if value is None]
        return board.make_move(choice(empty_spots))

    @staticmethod
    def reward(board):
        if not board.terminal:
            raise RuntimeError(f"reward called on nonterminal board {board}")
        if board.turn != board.winner:  # Corrected logical comparison
            return 0
        if board.winner is None:
            return 0.5
        raise RuntimeError(f"board has unknown winner type{board.winner}")

    @staticmethod
    def is_terminal(board):
        return board.terminal

    @staticmethod
    def make_move(board, index):
        tup = board.tup[:index] + (board.turn,) + board.tup[index + 1:]
        turn = not board.turn
        winner = TicTacToeBoard.find_winner(tup)
        is_terminal = (winner is not None) or not any(v is None for v in tup)
        return TicTacToeBoard(tup, turn, winner, is_terminal)

    @staticmethod
    def display_board(board):
        to_char = lambda v: "x" if v is True else ("0" if v is False else " ")
        rows = [[to_char(board.tup[3 * row + col]) for col in range(3)] for row in range(3)]
        return ("\n 123\n" + "\n".join(str(i+1) + " " + " ".join(row) for i, row in enumerate(rows)) + "\n")

    @staticmethod
    def play_game():
        # tree = MCTS()  # Assuming MCTS is defined elsewhere
        board = TicTacToeBoard.new_Board()
        print(TicTacToeBoard.display_board(board))
        while True:
            row_col = input("Enter position as row,col: ")
            row, col = map(int, row_col.split(","))
            index = 3 * (row - 1) + (col - 1)
            if board.tup[index] is not None:
                raise RuntimeError("Invalid move")
            board = TicTacToeBoard.make_move(board, index)
            print(TicTacToeBoard.display_board(board))
            if board.terminal:
                print('Game Over')
                break

            # Example loop, assuming MCTS and tree are defined
            # for _ in range(50):
            #     tree.do_rollout(board)
            # board = tree.choose(board)
            # print(TicTacToeBoard.display_board(board))
            # if board.terminal:
            #     print('Game Over')
            #     break

    @staticmethod
    def winning_combos():
        for start in range(0, 9, 3):
            yield (start, start + 1, start + 2)
        for start in range(3):
            yield (start, start + 3, start + 6)
        yield (0, 4, 8)
        yield (2, 4, 6)

    @staticmethod
    def find_winner(tup):
        for i1, i2, i3 in TicTacToeBoard.winning_combos():
            v1, v2, v3 = tup[i1], tup[i2], tup[i3]
            if False == v1 == v2 == v3:  # Corrected logical comparison
                return False
            if True == v1 == v2 == v3:  # Corrected logical comparison
                return True
        return None

    @staticmethod
    def new_Board():
        return TicTacToeBoard(tup=(None,) * 9, turn=True, winner=None, terminal=False)

if __name__ == "__main__":
    TicTacToeBoard.play_game()
